In [1]:
# only when open in colab
!git init .
!git remote add origin https://github.com/coyote-bergstein/compression-using-neural-networks.git
!git pull origin speech-commands-prev

Reinitialized existing Git repository in /home/jovyan/work/.git/
fatal: detected dubious ownership in repository at '/home/jovyan/work'
To add an exception for this directory, call:

	git config --global --add safe.directory /home/jovyan/work
fatal: detected dubious ownership in repository at '/home/jovyan/work'
To add an exception for this directory, call:

	git config --global --add safe.directory /home/jovyan/work


In [9]:
!pip install pydub

!pip install numba==0.59.1 # fixes compability issue
!pip install tensorflow-datasets librosa


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 3.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.2 MB/s eta 0:00:00a 0:00:01


In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import IPython.display as ipd
import random
from tensorflow.keras import layers, models
from tensorflow.keras import backend

In [6]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

In [7]:
n_mels = 128

In [10]:
# only when open in colab
import importlib
import source
from source.utils import generate_spectrogram, reconstruct_audio, normalize_standard
from source.constants import HOP_LENGTH, N_FFT, AUDIO_SAMPLE_RATE, WIN_LENGTH, WINDOW, CENTER, PAD_MODE, POWER, N_ITER, BATCH_SIZE, EPOCHS, OPTIMIZER, LOSS
importlib.reload(source.constants)
importlib.reload(source.utils)

<module 'source.utils' from '/home/jovyan/work/source/utils.py'>

## Dataset

In [ ]:
train_ds = tfds.load('speech_commands', split='train')

2024-06-08 15:49:54.301458: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

In [ ]:
for audio in train_ds.take(1):
    print(audio)

In [ ]:
train_ds = train_ds.map(lambda x: x['audio'])

In [ ]:
def get_min_max_length(dataset):
    min_length = float('inf')
    max_length = 0
    for audio in dataset:
        length = audio.shape[0]
        if length < min_length:
            min_length = length
        if length > max_length:
            max_length = length
    return min_length, max_length

In [ ]:
min_length, max_length = get_min_max_length(train_ds)
print(f'Min length: {min_length}, max length: {max_length}')

## Audio processing testing

In [ ]:
for audio in train_ds.take(5):
    audio_normalized, mean, std = normalize_standard(audio.numpy())
    print("Audio shape:", audio_normalized.shape)
    S = generate_spectrogram(signal=audio_normalized, sr=AUDIO_SAMPLE_RATE, hop_length=HOP_LENGTH, frame_size=N_FFT, win_length=WIN_LENGTH, window=WINDOW, center=CENTER, pad_mode=PAD_MODE, power=POWER)
    print("Spectrogram shape:", S.shape)
    reconstructed_signal = reconstruct_audio(S=S, sr=AUDIO_SAMPLE_RATE, hop_length=HOP_LENGTH, frame_size=N_FFT, mean=mean, std=std, win_length=WIN_LENGTH, window=WINDOW, center=CENTER, pad_mode=PAD_MODE, power=POWER, n_iter=N_ITER)
    
    print("Original Audio:")
    ipd.display(ipd.Audio(audio_normalized, rate=AUDIO_SAMPLE_RATE))
    
    print("Reconstructed Audio shape:", reconstructed_signal.shape)
    print("Reconstructed Audio:")
    ipd.display(ipd.Audio(reconstructed_signal, rate=AUDIO_SAMPLE_RATE))

## Padding audio to the maximum length

In [ ]:
def pad_audio(audio, length):
    audio_shape = tf.shape(audio)  # Get the dynamic shape of the audio tensor
    pad_length = length - audio_shape[0]  # Calculate the padding length
    padding = [[0, pad_length]]  # Create the padding specification as a list of lists
    padded_audio = tf.pad(audio, padding)  # Pad the audio tensor
    return padded_audio

padded_train_ds = train_ds.map(lambda x: pad_audio(audio=x, length=max_length))

In [ ]:
#min_length, max_length = get_min_max_length(padded_train_ds)
#print(f'Min length: {min_length}, max length: {max_length}')
#for audio in padded_train_ds: assert audio.shape[0] == max_length

In [ ]:
def preprocess(example):
    audio_normalized, mean, std = normalize_standard(example.numpy())
    S = generate_spectrogram(signal=audio_normalized, sr=AUDIO_SAMPLE_RATE, hop_length=HOP_LENGTH, frame_size=N_FFT, win_length=WIN_LENGTH, window=WINDOW, center=CENTER, pad_mode=PAD_MODE, power=POWER)
    S = np.expand_dims(S, axis=-1)
    return S.astype(np.float32)

def preprocess_with_py_function(example):
    audio = tf.py_function(preprocess, [example], [tf.float32])
    # audio.set_shape([n_mels, None])
    return audio

In [ ]:
ds_train = padded_train_ds.map(preprocess_with_py_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.map(lambda x: (x, x))
ds_train = ds_train.batch(BATCH_SIZE).cache().prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
for audio, label in ds_train.take(10):
    print(audio.shape)

In [ ]:
#for audio, label in ds_train:
#    assert audio.shape[0] == BATCH_SIZE and audio.shape[3] == 1

## Define model architecture

### Current network

In [ ]:
# Define the Encoder
def build_encoder(input_shape):
    encoder_input = layers.Input(shape=input_shape, name='encoder_input')

    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(encoder_input)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)

    encoder_output = layers.Flatten()(x)
    encoder_output = layers.Dense(128, activation='relu')(encoder_output)

    return models.Model(encoder_input, encoder_output, name='encoder')

# Define the Decoder
def build_decoder(encoded_shape):
    decoder_input = layers.Input(shape=(encoded_shape,), name='decoder_input')

    x = layers.Dense(16 * 16 * 32, activation='relu')(decoder_input)
    x = layers.Reshape((16, 16, 32))(x)
    x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)

    # Add a Cropping layer to match the exact input shape
    x = layers.Cropping2D(cropping=((0, 0), (0, 2)))(x)

    decoder_output = layers.Conv2DTranspose(1, (3, 3), activation='linear', padding='same')(x)

    return models.Model(decoder_input, decoder_output, name='decoder')

# Define the Autoencoder
def build_autoencoder(input_shape):
    encoder = build_encoder(input_shape)
    decoder = build_decoder(encoder.output_shape[1])

    autoencoder_input = layers.Input(shape=input_shape, name='autoencoder_input')
    encoded = encoder(autoencoder_input)
    decoded = decoder(encoded)

    return models.Model(autoencoder_input, decoded, name='autoencoder')

# Input shape for the spectrograms
input_shape = (128, 126, 1)  # Example shape, modify as needed

# Build and compile the autoencoder
autoencoder = build_autoencoder(input_shape)
autoencoder.compile(optimizer=OPTIMIZER, loss=LOSS)

# Summary of the model
autoencoder.summary()

### Previous network

In [ ]:
input_shape = (n_mels, 126, 1)

# Encoder
encoder_input = layers.Input(shape=input_shape)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(encoder_input)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
shape_before_dense = backend.int_shape(x)[1:]
x = layers.Flatten()(x)
encoded = layers.Dense(20)(x)

encoder = tf.keras.Model(encoder_input, encoded, name="encoder")

# Decoder
decoder_input = layers.Input(shape=20, name="decoder_input")
x = layers.Dense(np.prod(shape_before_dense), name="decoder_dense")(decoder_input)
x = layers.Reshape(target_shape=shape_before_dense)(x)
x = layers.Conv2DTranspose(8, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2DTranspose(8, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2DTranspose(16, (3, 3), activation='relu', padding='same')(x)
decoded = layers.Conv2DTranspose(1, (3, 3), activation='relu', padding='same')(x)

decoder = tf.keras.Model(decoder_input, decoded, name="decoder")

autoencoder = tf.keras.Model(encoder_input, decoder(encoder(encoder_input)), name="autoencoder")
autoencoder.compile(optimizer=OPTIMIZER, loss=LOSS)
autoencoder.summary()

### Training

In [ ]:
history = autoencoder.fit(ds_train, epochs=5, verbose=1)

In [25]:
autoencoder.save('autoencoder_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [27]:
ds_predict = train_ds.map(preprocess_with_py_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [48]:
# Test the autoencoder reconstruction
for example in ds_predict.take(5):

    print("Original Audio:")
    reconstructed_signal_uncompressed = reconstruct_audio(np.squeeze(example[0]), HOP_LENGTH, N_FFT)

    # Use autoencoder to compress and decompress the audio
    S_compressed = autoencoder.predict(np.array(example))
    reconstructed_signal_compressed = reconstruct_audio(np.squeeze(S_compressed), HOP_LENGTH, N_FFT)

    print("Original Audio:")
    ipd.display(ipd.Audio(reconstructed_signal_uncompressed, rate=AUDIO_SAMPLE_RATE))

    print("Reconstructed Audio:")
    ipd.display(ipd.Audio(reconstructed_signal_compressed, rate=AUDIO_SAMPLE_RATE))


Original Audio:
1/1 [==============================] - 0s 19ms/step
Original Audio:


Reconstructed Audio:


Original Audio:
1/1 [==============================] - 0s 18ms/step
Original Audio:


Reconstructed Audio:


Original Audio:
1/1 [==============================] - 0s 18ms/step
Original Audio:


Reconstructed Audio:


Original Audio:
1/1 [==============================] - 0s 18ms/step
Original Audio:


Reconstructed Audio:


Original Audio:
1/1 [==============================] - 0s 19ms/step
Original Audio:


Reconstructed Audio:
